# Data ADS

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

In [2]:
# 1. Đọc danh sách ASIN từ file txt
with open('ASIN.txt', 'r') as f:
    asin_list = [line.strip() for line in f.readlines()]

# 2. Tạo list chứa các file .xlsx trong thư mục Data
folder_path = 'Data_ADS'
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# 3. Đọc từng file, xử lý rồi append vào list
all_data = []

for file in files:
    # Trích xuất ngày từ tên file
    date_str = file.split('_')[1].replace('.xlsx', '')  # "20250713"
    date_obj = datetime.strptime(date_str, "%Y%m%d")     # datetime object
    df = pd.read_excel(os.path.join(folder_path, file))
    
    # Thêm cột date định dạng chuẩn
    df['date'] = date_obj
    
    # Tạo cột ASIN từ 10 ký tự đầu tiên của cột Portfolio
    df['ASIN'] = df['Portfolio'].astype(str).str[:10]
    
    all_data.append(df)

# 4. Gộp tất cả thành một DataFrame
merged_df = pd.concat(all_data, ignore_index=True)

# 5. Lọc chỉ giữ lại các dòng có ASIN nằm trong danh sách
filtered_df = merged_df[merged_df['ASIN'].isin(asin_list)]

#6. Di chuuyển cột ASIN lên đầu, sau Campaign type	
filtered_df = filtered_df[['Campaign type', 'ASIN'] + [col for col in filtered_df.columns if col not in ['Campaign type', 'ASIN']]]

# ✅ Kết quả cuối cùng
filtered_df.head()

,Campaign type,ASIN,Campaign,Status,Country,Profile,Portfolio,Target type,Daily Budget,Current Budget,...,CPA,Sales Same SKU,Sales Other SKU,Orders Same SKU,Orders Other SKU,Units Same SKU,Units Other SKU,date,Top-of-search IS,Avg.time in Budget
2,sponsoredProducts,B0DH86X93V,B0DH86X93V_20oz_vintage 1985 weird navy_asin e...,Delivering,US,NewEleven,B0DH86X93V_TUMBLER 20_VINTAGE 1985 WEIRD BEING...,manual,$30.00,$0.00,...,5.25,119.88,0.0,6,--,6,--,2025-07-12,NaN,NaN
35,sponsoredProducts,B08R8R2LQF,B08R8R2LQF_20oz_May the forties black_40th bir...,Delivering,US,NewEleven,B08R8R2LQF_TUMBLER 20_MAY THE FORTIES BE WITH ...,manual,$10.00,$0.00,...,18.20,22.98,0.0,1,--,1,--,2025-07-12,NaN,NaN
38,sponsoredProducts,B08R8R2LQF,[Forties] all keyword,Delivering,US,NewEleven,B08R8R2LQF_TUMBLER 20_MAY THE FORTIES BE WITH ...,manual,$12.00,$0.00,...,5.83,68.94,0.0,3,--,3,--,2025-07-12,NaN,NaN
53,sponsoredProducts,B09XXP5V2W,"B09XXP5V2W_20oz_Quitter black_retirement gift_b,p",Delivering,US,NewEleven,B09XXP5V2W_TUMBLER 20_QUITTER BLACK_TRINH,manual,$10.00,$0.00,...,14.78,19.98,0.0,1,--,1,--,2025-07-12,NaN,NaN
69,sponsoredProducts,B0DH87FYNN,B0DH87FYNN_20oz_Vintage 1965 weird navy_60th b...,Delivering,US,NewEleven,B0DH87FYNN_TUMBLER 20_VINTAGE 1965 WEIRD BEING...,manual,$20.00,$0.00,...,6.60,39.96,0.0,2,--,2,--,2025-07-12,NaN,NaN


In [3]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("c:/Users/admin1/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1n2Ug3joJwsuuSsrx6T92Zi_dihht7KzIebRTGpKnehc"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("DATA_XN_ADS")

sheet1.clear()  # Xoá nội dung cũ trong sheet
set_with_dataframe(sheet1, filtered_df)

# Total GMV

In [4]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials


In [5]:
# 1. Đọc danh sách ASIN từ file txt
with open('ASIN.txt', 'r') as f:
    asin_list = [line.strip() for line in f.readlines()]

# 2. Tạo list chứa các file .csv trong thư mục Data
folder_path = 'Data_Total'
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 3. Đọc từng file, xử lý rồi append vào list
all_data = []

for file in files:
    # ✅ Dùng regex để tìm ngày theo pattern dd_mm_yyyy
    match = re.search(r'(\d{2}_\d{2}_\d{4})', file)
    
    if match:
        date_str = match.group(1)  # ví dụ: "20_07_2025"
        date_obj = datetime.strptime(date_str, "%d_%m_%Y")  # chuyển thành datetime object

        df = pd.read_csv(os.path.join(folder_path, file), sep=";")
        
        # Thêm cột 'Date' chuẩn hóa theo yyyy-mm-dd
        df['Date'] = date_obj.strftime('%Y-%m-%d')
        
        all_data.append(df)
    else:
        print(f"⚠️ Không tìm thấy ngày trong tên file: {file}")

# 4. Gộp tất cả thành một DataFrame
merged_df = pd.concat(all_data, ignore_index=True)

# ✅ 5. Chỉ giữ lại các dòng có ASIN nằm trong danh sách
merged_df = merged_df[merged_df['ASIN'].isin(asin_list)]

# 6. Ở cột Ads, Sales và Net profit, thay thế dấu phẩy bằng dấu chấm
merged_df['Ads'] = merged_df['Ads'].str.replace(',', '.').astype(float)
merged_df['Sales'] = merged_df['Sales'].str.replace(',', '.').astype(float)
merged_df['Net profit'] = merged_df['Net profit'].str.replace(',', '.').astype(float)

In [6]:
merged_df.head(7)

,Product,ASIN,SKU,Units,Refunds,Sales,Promo,Ads,Sponsored products (PPC),Sponsored Display,...,Net profit,Estimated payout,Expenses,Margin,ROI,BSR,Real ACOS,Sessions,Unit Session Percentage,Date
9,NewEleven 40th Birthday Gifts For Men Women - ...,B08R8R2LQF,JC-JMFX-W7IY,17.0,NaN,390.66,"-2,3",-38.98,"-38,98",NaN,...,147.13,"214,49",NaN,"37,66","205,57",19 693,"9,98",25.0,"68,00",2025-08-01
21,NewEleven Retirement Gifts For Men Women 2025 ...,B09XXP5V2W,FL-GKXG-L8RH,7.0,NaN,139.86,NaN,-22.11,"-22,11",NaN,...,36.69,"71,41",NaN,"26,23","105,67",25 371,"15,81",32.0,"21,88",2025-08-01
23,NewEleven Engagement Gifts for Couples 2025 - ...,B0DRT3MK34,87-ARXM-YV1I,7.0,NaN,69.93,NaN,-30.60,"-30,6",NaN,...,-2.74,"9,86",NaN,"-3,92","-21,75",91 538,"43,76",22.0,"31,82",2025-08-01
29,NewEleven Coworker Leaving Gifts - Farewell Gi...,B092HNBJT6,W7-BHYY-1NZC,6.0,NaN,119.88,-2,-41.13,"-41,13",NaN,...,4.60,"32,38",NaN,"3,84","16,56",18 419,"34,31",66.0,"9,09",2025-08-01
30,NewEleven Coworker Leaving Gifts - Gifts For C...,B0BRB6NJCT,MF-GYAU-GTQX,6.0,1.0,89.88,NaN,-25.47,"-25,47",NaN,...,-2.87,"9,23",NaN,"-3,19","-19,77",14 778,"28,34",82.0,"7,32",2025-08-01
33,NewEleven 60th Birthday Gifts For Men Women - ...,B0DH87FYNN,UM-NFPU-HUFM,6.0,1.0,119.88,NaN,-28.75,"-28,75",NaN,...,9.23,"29,09",NaN,"7,70","46,48",2 183,"23,98",31.0,"19,35",2025-08-01
42,NewEleven Gifts For New Mom 2025 – Pregnancy G...,B0DCJM56W3,SG-SV6L-3ILW,5.0,NaN,64.95,NaN,-35.63,"-35,63",NaN,...,-9.27,"-0,35",NaN,"-14,27","-83,14",9 129,"54,86",35.0,"14,29",2025-08-01


In [7]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 567 entries, 9 to 6362
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Product                   567 non-null    object 
 1   ASIN                      567 non-null    object 
 2   SKU                       560 non-null    object 
 3   Units                     476 non-null    float64
 4   Refunds                   59 non-null     float64
 5   Sales                     475 non-null    float64
 6   Promo                     45 non-null     object 
 7   Ads                       346 non-null    float64
 8   Sponsored products (PPC)  346 non-null    object 
 9   Sponsored Display         0 non-null      object 
 10  Sponsored brands (HSA)    0 non-null      object 
 11  Sponsored Brands Video    0 non-null      object 
 12  Google ads                0 non-null      float64
 13  Facebook ads              0 non-null      float64
 14  % Refunds     

In [8]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("c:/Users/admin1/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1n2Ug3joJwsuuSsrx6T92Zi_dihht7KzIebRTGpKnehc"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("DATA_SB_TOTAL")

sheet1.clear()  # Xóa dữ liệu cũ trước khi ghi mới
set_with_dataframe(sheet1, merged_df)